In [1]:
import os
import numpy as np
import random
import string
from matplotlib import pyplot as plt

In [2]:
import cv2

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)

In [ ]:
!pip uninstall opencv-python